In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 5.3 MB/s eta 0:00:00


In [ ]:
!pip install feature_engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.6/378.6 kB 5.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import warnings
import joblib
import plotly.express as px
from google.colab import drive
from datetime import date, timedelta
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import train_test_split
# from plot_metric.functions import BinaryClassification
from datetime import datetime
import random
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import DBSCAN, MeanShift
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import silhouette_score, davies_bouldin_score

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 500

from google.colab import auth
import gspread
from google.auth import default

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [ ]:
import json
data = {
  "account": "",
  "client_id": "764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com",
  "client_secret": "d-FL95Q19q7MQmFpd7hHD0Ty",
  "quota_project_id": "dev-sd-lake",
  "refresh_token": "1//05nz9IRnR1e2JCgYIARAAGAUSNwF-L9Ir_zi7EkfNFJb02e5XxL3e4Qicf7BN708CjMo66DuPPMhkd09jS6lIra-zubUPFGORcyU",
  "type": "authorized_user",
  "universe_domain": "googleapis.com"
}
json_data = json.dumps(data)
!echo '{json_data}' > '/content/.config/application_default_credentials.json'

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project='dev-sd-lake')

category_total_shelf_life_query = """
SELECT * FROM pollen.category_total_shelf_life;
"""

category_total_shelf_life = client.query_and_wait(category_total_shelf_life_query).to_dataframe()
category_total_shelf_life = category_total_shelf_life.rename(columns=category_total_shelf_life.iloc[0]).iloc[1:].reset_index(drop = True)

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project='dev-sd-lake')

product_listings_query = """
SELECT * FROM pollen.product_listings;
"""

product_listings = client.query_and_wait(product_listings_query).to_dataframe()


client = bigquery.Client(project='dev-sd-lake')

products_query = """
SELECT * FROM pollen.products;
"""

products = client.query_and_wait(products_query).to_dataframe()


client = bigquery.Client(project='dev-sd-lake')

product_categories_query = """
SELECT * FROM pollen.product_categories;
"""

product_categories = client.query_and_wait(product_categories_query).to_dataframe()


client = bigquery.Client(project='dev-sd-lake')

product_subcategories_query = """
SELECT * FROM pollen.product_subcategories;
"""

product_subcategories = client.query_and_wait(product_subcategories_query).to_dataframe()

client = bigquery.Client(project='dev-sd-lake')

sellers_query = """
SELECT * FROM pollen.sellers;
"""

sellers = client.query_and_wait(sellers_query).to_dataframe()

In [ ]:
offers_query = """
SELECT * FROM pollen.offers;
"""

offers = client.query_and_wait(offers_query).to_dataframe()

In [ ]:
orders_level_1_query = """
SELECT * FROM pollen.orders_level_1;
"""

orders_level_1 = client.query_and_wait(orders_level_1_query).to_dataframe()

In [ ]:
orders_level_2_query = """
SELECT * FROM pollen.orders_level_2;
"""

orders_level_2 = client.query_and_wait(orders_level_2_query).to_dataframe()

In [ ]:
sellers = sellers[['seller_name', 'persona_seller_type']].drop_duplicates()

In [ ]:
print(product_listings.shape, products.shape, product_categories.shape, product_subcategories.shape, sellers.shape)

(182283, 41) (182291, 9) (30, 3) (223, 4) (109, 2)


In [ ]:
def get_order_date(df_order_sheet1, df_order_sheet2):
  order_date_to_tracking_no_mapping = df_order_sheet1[['tracking_no', 'order_date']].drop_duplicates()
  df_order_sheet2 = pd.merge(df_order_sheet2, order_date_to_tracking_no_mapping, 'left',  on = ['tracking_no'])
  df_order_sheet2.drop('date_of_order', axis = 1, inplace = True)
  df_order = df_order_sheet2.copy()
  return df_order

orders = get_order_date(orders_level_1, orders_level_2)

In [ ]:
orders = orders.drop_duplicates(subset = ['sku_number', 'expiry_date', 'tracking_no'])
offers = offers.drop_duplicates(subset = ['sku_number', 'expiry_date', 'tracking_no'])
offers = offers.rename(columns = {'total_units' : 'total_units_offered'})
orders = orders.rename(columns = {'total_units' : 'total_units_ordered', 'total_offer_price_usd' : 'total_order_price_usd'})

offers = pd.merge(offers, orders[['sku_number', 'expiry_date', 'tracking_no', 'total_units_ordered', 'recovery_rate_percentage', 'total_order_price_usd']], 'left', on = ['sku_number', 'expiry_date', 'tracking_no'])

In [ ]:
product_subcategories = product_subcategories.rename(columns = {'id' : 'subcategory_id', 'name' : 'product_subcategory'})
product_categories = product_categories.rename(columns = {'id' : 'category_id', 'name' : 'product_category'})

products = pd.merge(products, product_categories, 'left', on = 'category_id')
products = pd.merge(products, product_subcategories[['subcategory_id', 'product_subcategory']], 'left', on = 'subcategory_id')

In [ ]:
products = products.rename(columns = {'id' : 'product_id'})
products = products.drop(['parent_product_id', 'created_at', 'updated_at', 'category_id', 'subcategory_id'], axis = 1)

product_listings = product_listings.drop(['created_at', 'updated_at', 'pollen_updated_price_per_unit_local',
                       'barcode', 'barcode_key', 'manufactured_date', 'batch_number', 'image_links', 'usd_conversion', 'scoring'], axis = 1)
product_listings = pd.merge(product_listings, products, 'left', on = 'product_id')

In [ ]:
product_listings.head(2)

,id,product_id,seller_id,record_id,seller_name,expiry_date,package_type,qty_of_carton,units_per_package_type,total_number_of_items,currency,retail_price_per_unit_local,asking_price_per_unit_local,pack_size_number,pack_size_unit,cbm_carton,length,breadth,height,unit_dimensions,weight_per_carton_kg,warehouse_address,warehouse_country,country_of_origin,package_labeled_language,product_restriction,priority,retail_price_per_unit_usd,asking_price_per_unit_usd,inventory_class,updated_on,sku_number,sku_product_name,brand,product_category,description,product_subcategory
0,2d61c777-e0d5-4744-b9eb-dd24f74ee061,44642cbd-8752-4f55-a368-669f22e9e8f8,SE1712631761502,SE1712631761502G4311200,L'Oreal Indonesia,2025-01-01,Carton,64.20,60.0,3852.0,IDR,24900.0,5991.5,31.0,g,0.01,254.0,156.0,219.0,MM,2.24,Bekasi,Indonesia,-,-,NaN,P2,1.53,0.37,CORE,2024-05-28,G4311200,GRN 2 STEP PW MASK B5 GOJI EB,GARNIER,SKIN CARE,SKIN CARE,MASKS & EXFOLIATORS
1,41508680-7c85-47aa-a54c-2bcbbddb56e9,6ca31813-8cac-4459-a53e-d68b62bf337d,SE1712631761502,SE1712631761502G4218000,L'Oreal Indonesia,2025-01-28,Carton,58.43,60.0,3506.0,IDR,23400.0,6552.7,19.7,g,0.01,185.0,150.0,197.0,MM,1.42,Bekasi,Indonesia,-,-,NaN,P2,1.44,0.40,CORE,2024-05-28,G4218000,GRN 5 MIN AMP MASK WATERMELON EB,GARNIER,SKIN CARE,SKIN CARE,MASKS & EXFOLIATORS


In [ ]:
product_listings.inventory_class.value_counts()

,count
inventory_class,
NON_CORE,115580
CORE,66703


In [ ]:
df_inv = product_listings[product_listings.inventory_class == 'CORE']
non_core_inv = product_listings[product_listings.inventory_class == 'NON_CORE']
df_inv = df_inv.reset_index(drop = True)
non_core_inv = non_core_inv.reset_index(drop = True)

In [ ]:
cols_list_underscore_cleaning = ['sku_product_name', 'brand', 'product_category', 'product_subcategory', 'warehouse_country',
            'country_of_origin', 'pack_label_language', 'product_restricted_countries', 'shelf_life_bucket']

inv_cols_list_to_change_dtypes = ['qty_of_cartons', 'units_per_cartons', 'retail_price_per_case_(local)', 'retail_price_per_case_(usd)', 'asking_price_per_case_(local)',
                              'asking_price_per_case_(usd)', 'total_retail_price_(local)', 'total_retail_price_(usd)', 'total_asking_price_(local)',
                              'total_asking_price_(usd)', 'pack_size_(number)', 'total_cbm', 'cbm_per_case', 'package_dimensions_per_package_type_(length)',
                              'package_dimensions_per_case_(width)', 'package_dimensions_per_case_(height)', 'net_weight_\nper_unit(kg)', 'net_weight_per_case_(kg)',
                              'gross_weight_per_case_(kg)', 'total_net_weight_(kg)', 'total_gross_weight_(kg)', 'cases_per_pallet', 'number_of_pallets', 'discount',
                              'shelf_remaining_days']

record_attributes = ['sku_number', 'sku_name', 'brand', 'product_category', 'product_subcategory', 'retail_price_per_unit_local', 'currency']

n_days_before_date = pd.to_datetime(date.today() - timedelta(days = 14))

THRESHOLD = 0.65

In [ ]:
def dropping_null_brand_cat_subcats(df_inv):
  df_inv = df_inv[~df_inv.brand.isnull()]
  df_inv = df_inv[~df_inv.product_category.isnull()]
  df_inv = df_inv[~df_inv.product_subcategory.isnull()]
  return df_inv

def cols_to_lower_rem_space(df, cols_list):
  for col in ['sku_name', 'product_name', 'brand', 'brand_', 'product_description', 'product_category', 'product_sub_category', 'product_subcategory',\
              'warehouse_location', 'buyer', 'seller', 'package_type', 'deal_type', 'origin_', 'destination', 'region_of_export', \
              'domestic_export', 'order_type', 'country', 'currency', 'country_of_origin', 'pack_label_language', 'dangerous_goods\n(y/n)', 'shelf_life_bucket',
              'relavant_(r)/ir-relavant(ir)', 'product_restricted_countries', 'store_name', 'quarter', 'temp_reference', 'tracking_no', 'offer_type', 'fiscal_month',
              'proposed_buyer_product_preferences', 'product_shelf_life_months', 'priority', 'seller', 'lms_seller_id', 'persona', 'core_vs._\nnon_core', 'sku_product_name',
              'warehouse_country', 'warehouse_address', 'measurement_units', 'listing_currency', 'manufacturing_country', 'lbh_measurement_units', 'package_type', 'seller_name',
              'pack_size_unit', 'package_labeled_language', 'inventory_class', 'description', 'unit_dimensions']:
              if col in cols_list:
                df[col] = df[col].str.lower().str.replace(' ', '_')
  return df

def remove_starting_and_trailing_underscores(x):
  if x != '':
    if x[0] == '_':
      for i in range(len(x)):
        if x[i] != '_':
          start = i
          break
      x = x[start:]
    if x[-1] == '_':
      for i in reversed(range(len(x))):
        if x[i] != '_':
          end = i
          break
      x = x[:end+1]
  return x

def cleaning_underscores_inv_data(df_inv, col_list):
  for col in col_list:
    print(col)
    df_inv[col] = df_inv[col].apply(remove_starting_and_trailing_underscores)
  return df_inv

def drop_dups_and_clean(df_inv):
  print('Dropping above duplicates ...')
  df_inv = df_inv.drop_duplicates()
  return df_inv

In [ ]:
df_inv = dropping_null_brand_cat_subcats(df_inv)
df_inv = cols_to_lower_rem_space(df_inv, df_inv.columns)
df_inv = drop_dups_and_clean(df_inv)

non_core_inv = cols_to_lower_rem_space(non_core_inv, non_core_inv.columns)

Dropping above duplicates ...


# Non Core

In [ ]:
non_core_inv.head(2)

,id,product_id,seller_id,record_id,seller_name,expiry_date,package_type,qty_of_carton,units_per_package_type,total_number_of_items,currency,retail_price_per_unit_local,asking_price_per_unit_local,pack_size_number,pack_size_unit,cbm_carton,length,breadth,height,unit_dimensions,weight_per_carton_kg,warehouse_address,warehouse_country,country_of_origin,package_labeled_language,product_restriction,priority,retail_price_per_unit_usd,asking_price_per_unit_usd,inventory_class,updated_on,sku_number,sku_product_name,brand,product_category,description,product_subcategory
0,b61c2bb9-f203-4447-bd21-34c8b2d8a03a,c741e020-cae7-4258-b71f-3bc338b38412,SE1712631761542,SE171263176154204R3I1MKJPD97H7OBZ60UTU1Z,acommerce_singapore,2023-05-31,pieces,NaN,NaN,6.0,sgd,19.9,1.99,NaN,None,0.0,10.0,10.0,10.0,cm,0.50,sg-acom01,None,None,None,NaN,p1,15.116504,1.511650,non_core,2023-01-25,04R3I1MKJPD97H7OBZ60UTU1Z,l'oreal_paris_color_riche_satin_105_nude_in_paris,l'oreal_paris,makeup,makeup,lip_makeup
1,329218a0-49c7-4e36-bfe7-e10d17de8dfc,0cec2d8c-024a-4929-87fa-358f264016b7,SE1712631761542,SE171263176154211YAP8LR1W3OQSFJGWCLTD1DQ,acommerce_singapore,2023-05-31,pieces,NaN,NaN,160.0,sgd,2.0,0.20,NaN,None,0.0,8.0,5.0,0.2,cm,0.15,sg-acom02,None,None,None,NaN,p1,1.519247,0.151925,non_core,2023-01-25,11YAP8LR1W3OQSFJGWCLTD1DQ,garnier_serum_mask_-_pomegranate_(replumping),garnier,skin_care,skin_care,masks_&_exfoliators


In [ ]:
non_core_inv = non_core_inv.rename(columns = {'total_number_of_items' : 'total_units'})
non_core_inv['updated_on'] = pd.to_datetime(non_core_inv['updated_on'])
non_core_inv['expiry_date'] = pd.to_datetime(non_core_inv['expiry_date'])

In [ ]:
print(non_core_inv.shape)
non_core_inv = non_core_inv.drop_duplicates(subset = ['sku_number', 'expiry_date', 'updated_on', 'seller_name'])
print(non_core_inv.shape)

(115580, 37)
(64433, 37)


In [ ]:
offers = offers.rename(columns = {'seller' : 'seller_name'})
offers['date_of_offer'] = pd.to_datetime(offers['date_of_offer'])
offers['seller_name'] = offers['seller_name'].str.lower()
offers['total_units_ordered'] = offers['total_units_ordered'].str.replace(',', '')
offers['total_units_ordered'] = offers['total_units_ordered'].astype(float)

In [ ]:
def merge_non_core_inv_and_offer(non_core_inv, df_offer):
  temp = pd.merge(non_core_inv, df_offer[['sku_number', 'seller_name', 'date_of_offer', 'actual', 'tracking_no', 'buyer', 'total_units_ordered',
                                          'recovery_rate_percentage', 'offer_price_per_unit_usd', 'total_order_price_usd', 'destination']], 'left', on = ['sku_number', 'seller_name'])
  # temp['updated_on'] = pd.to_datetime(temp['updated_on'])
  temp['offer_day_diff_updated_inv'] = (temp['date_of_offer'] - temp['updated_on']).dt.days
  # avoiding future entry from same sku and seller to get filtered out hence nulling it as will remain in the dataframe
  temp['offer_day_diff_updated_inv'] = np.where(temp.offer_day_diff_updated_inv < 0, np.nan, temp['offer_day_diff_updated_inv'])
  temp = temp[(temp.offer_day_diff_updated_inv >= 0) | (temp.offer_day_diff_updated_inv.isnull())]
  df_non_core_inv_min_offer_day_diff_updated_inv = temp.groupby(['sku_number', 'seller_name', 'date_of_offer']).agg({'offer_day_diff_updated_inv' : 'min'}).reset_index()\
                                        .rename(columns = {'offer_day_diff_updated_inv' : 'min_offer_day_diff_updated_inv'})
  temp = pd.merge(temp, df_non_core_inv_min_offer_day_diff_updated_inv, 'left', on = ['sku_number', 'seller_name', 'date_of_offer'])
  temp = temp[(temp.offer_day_diff_updated_inv == temp.min_offer_day_diff_updated_inv) | (temp.offer_day_diff_updated_inv.isnull())]
  temp.drop('min_offer_day_diff_updated_inv', axis = 1, inplace = True)
  return temp

In [ ]:
non_core_inv = merge_non_core_inv_and_offer(non_core_inv, offers)
# non_core_inv['sell_thru_rate'] = non_core_inv['total_units_ordered'] / non_core_inv['total_units']
# non_core_inv['sell_thru_rate'] = np.where(non_core_inv['sell_thru_rate'] > 1, 1, non_core_inv['sell_thru_rate'])

In [ ]:
non_core_inv['shelf_life_remaining_days'] = (non_core_inv['expiry_date'] - non_core_inv['updated_on']).dt.days

In [ ]:
df_cluster = non_core_inv[['seller_name', 'warehouse_country', 'priority', 'retail_price_per_unit_usd', 'asking_price_per_unit_usd',
        'brand', 'product_category', 'product_subcategory']]

categorical_cols = ['priority', 'warehouse_country', 'brand', 'product_category', 'product_subcategory', 'seller_name']
df_encoded = df_cluster.copy()
le_dict = {}

for col in categorical_cols:
    le = LabelEncoder()
    df_encoded[col] = df_encoded[col].astype(str)  # convert NaNs to string "nan"
    df_encoded[col] = le.fit_transform(df_encoded[col])
    le_dict[col] = le

# Step 2: Apply IterativeImputer for missing values
imputer = IterativeImputer(random_state=42)
X_imputed = imputer.fit_transform(df_encoded)

# Step 3: Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)



# Step 4: Run DBSCAN
model = DBSCAN(eps=0.5, min_samples=200)
# model = MeanShift(bin_seeding = True)
df_cluster['cluster'] = model.fit_predict(X_scaled)

labels = df_cluster['cluster'].values
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

if n_clusters >= 2:
    # Remove noise points for silhouette score
    mask = labels != -1
    score = silhouette_score(X_scaled[mask], labels[mask])
    print(f"Silhouette Score (excluding noise): {score:.4f}")
else:
    print("Silhouette Score cannot be computed (requires at least 2 clusters excluding noise).")


non_core_inv['cluster'] = df_cluster['cluster']
non_core_inv['cluster'].unique()

Silhouette Score (excluding noise): 0.8634


array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23])

In [ ]:
temp = non_core_inv.groupby('cluster')['destination'].value_counts().rename('counts').reset_index()
temp = temp.groupby('cluster').destination.first().reset_index()
cluster_to_popular_destination_country = dict(zip(temp.cluster, temp.destination))
DEFAULT = non_core_inv['destination'].mode().values[0]
print(DEFAULT)


def set_destination(x):
  if pd.isna(x['destination']) and x['cluster'] in cluster_to_popular_destination_country:
    return cluster_to_popular_destination_country[x['cluster']]
  elif pd.isna(x['destination']) and x['cluster'] not in cluster_to_popular_destination_country:
    return DEFAULT
  else:
    return x['destination']

non_core_inv['destination'] = non_core_inv.apply(set_destination, axis = 1)
non_core_inv = non_core_inv.drop('cluster', axis = 1)

Indonesia


In [ ]:
temp = non_core_inv[['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', 'seller_name', 'updated_on', 'priority', 'total_units', 'shelf_life_remaining_days',
                     'warehouse_country', 'destination', 'total_units_ordered', 'recovery_rate_percentage']].sort_values(by = ['sku_number', 'brand', 'product_category', 'product_subcategory',
                      'expiry_date', 'seller_name', 'updated_on', 'priority', 'warehouse_country', 'destination'])
temp = temp.reset_index(drop = True)

In [ ]:
def get_prev_inv_occurence_next_to_current_for_non_core_inv(temp):
  new_col_names_for_shifted_inv = dict()
  for i in temp.columns:
    new_col_names_for_shifted_inv[i] = i+'_prev'

  temp_shifted = temp.shift(1)
  temp_shifted = temp_shifted.rename(columns = new_col_names_for_shifted_inv)
  temp = pd.concat([temp, temp_shifted], axis=1)
  return temp

# not taking expiry into account

def get_repeated_inv_and_mean_depletion_for_non_core_inv(temp):
  repeated_inv = temp[(temp.sku_number == temp.sku_number_prev) & (temp.seller_name == temp.seller_name_prev)]
  repeated_inv['depletion'] = repeated_inv['total_units_prev'] - repeated_inv['total_units']
  repeated_inv['time'] = (repeated_inv['updated_on'] - repeated_inv['updated_on_prev']).dt.days
  repeated_inv = repeated_inv.drop_duplicates(subset = ['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', \
                            'seller_name', 'updated_on', 'priority'])[['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', 'seller_name', 'updated_on_prev',\
                                                                        'priority', 'depletion', 'time', 'total_units_prev', 'total_units', 'priority_prev',\
                                                                        'shelf_life_remaining_days_prev', 'warehouse_country_prev', 'destination_prev',
                                                                       'total_units_ordered_prev', 'recovery_rate_percentage_prev']]

  return repeated_inv

## Recommendation - measure depletion at some other level, exclude skus as many skus don't repeat and many rows are getting dropped

In [ ]:
temp = get_prev_inv_occurence_next_to_current_for_non_core_inv(temp)
repeated_inv_non_core = get_repeated_inv_and_mean_depletion_for_non_core_inv(temp)
repeated_inv_non_core = repeated_inv_non_core.reset_index(drop = True)
repeated_inv_non_core['priority'] = np.where(repeated_inv_non_core.priority != repeated_inv_non_core.priority_prev, \
                                                   repeated_inv_non_core.priority_prev, repeated_inv_non_core.priority)

In [ ]:
# 90 days sales life cycle
repeated_inv_non_core = repeated_inv_non_core[repeated_inv_non_core.time <= 90]

mean_depletion_per_cat_non_core_inv = repeated_inv_non_core.groupby(['product_category']).depletion.mean().reset_index().sort_values(by = 'depletion')

low_depletion_cats = list(mean_depletion_per_cat_non_core_inv[mean_depletion_per_cat_non_core_inv['depletion'] <= mean_depletion_per_cat_non_core_inv.depletion.quantile(0.33)].product_category)
mid_depletion_cats = list(mean_depletion_per_cat_non_core_inv[mean_depletion_per_cat_non_core_inv['depletion'] <= mean_depletion_per_cat_non_core_inv.depletion.quantile(0.67)].product_category)
mid_depletion_cats = [i for i in mid_depletion_cats if i not in low_depletion_cats]
high_depletion_cats = list(mean_depletion_per_cat_non_core_inv[mean_depletion_per_cat_non_core_inv['depletion'] <= mean_depletion_per_cat_non_core_inv.depletion.quantile(1)].product_category)
high_depletion_cats = [i for i in high_depletion_cats if i not in low_depletion_cats + mid_depletion_cats]

print(low_depletion_cats, '\n', mid_depletion_cats, '\n', high_depletion_cats)

['beverages', 'health_&_supplments', 'industrial', 'oral_care', 'skin_care', 'home_decor', 'fashion_-_accessories'] 
 ['bedding_&_bath', 'furniture', 'kitchen_&_dining', 'home_-_other', 'home_-_decor', 'home_improvement_and_tools', 'other', 'baby'] 
 ['hair_care', 'home_care', 'fragrance', 'sports,_fitness,_&_outdoors', 'makeup', 'home', 'food']


In [ ]:
def set_depletion_non_core(row):
  if row['depletion'] < 0:
    return row['total_units_prev']
  elif row['product_category'] in high_depletion_cats and row['time'] <= 20:
    return row['depletion']
  elif row['product_category'] in high_depletion_cats and row['time'] > 20:
    return row['total_units_prev']
  elif row['product_category'] in mid_depletion_cats and row['time'] <= 45:
    return row['depletion']
  elif row['product_category'] in mid_depletion_cats and row['time'] > 45:
    return row['total_units_prev']
  elif row['product_category'] in low_depletion_cats and row['time'] <= 60:
    return row['depletion']
  elif row['product_category'] in low_depletion_cats and row['time'] > 60:
    return row['total_units_prev']

In [ ]:
repeated_inv_non_core['depletion'] = repeated_inv_non_core.apply(set_depletion_non_core, axis=1)

In [ ]:
def remove_percent_sign(x):
  if pd.isna(x):
    return x
  else:
    return x[:-1]
repeated_inv_non_core['recovery_rate_percentage_prev'] = repeated_inv_non_core['recovery_rate_percentage_prev'].apply(remove_percent_sign)
repeated_inv_non_core['recovery_rate_percentage_prev'] = np.where(repeated_inv_non_core['recovery_rate_percentage_prev'] == '', np.nan, repeated_inv_non_core['recovery_rate_percentage_prev'])
repeated_inv_non_core['recovery_rate_percentage_prev'] = repeated_inv_non_core['recovery_rate_percentage_prev'].astype(float)
repeated_inv_non_core['recovery_rate_percentage_prev'] = np.where(repeated_inv_non_core['recovery_rate_percentage_prev'].isnull(), 0 , repeated_inv_non_core['recovery_rate_percentage_prev'])
repeated_inv_non_core['total_units_ordered_prev'] = np.where(repeated_inv_non_core['total_units_ordered_prev'].isnull(), 0, repeated_inv_non_core['total_units_ordered_prev'])

In [ ]:
repeated_inv_non_core = repeated_inv_non_core.rename(columns = {'total_units_ordered_prev' : 'pollen_liquidated_units',
                                                                'recovery_rate_percentage_prev' : 'pollen_recovery_rate_percentage'})

In [ ]:
repeated_inv_non_core.sample(2)

,sku_number,brand,product_category,product_subcategory,expiry_date,seller_name,updated_on_prev,priority,depletion,time,total_units_prev,total_units,priority_prev,shelf_life_remaining_days_prev,warehouse_country_prev,destination_prev,pollen_liquidated_units,pollen_recovery_rate_percentage
2012,E2S06133,home,home_improvement_and_tools,women's_tops,NaT,excess2sell,2025-03-03,p3,2.0,9,2.0,11.0,p3,NaN,india,Indonesia,0.0,0.0
1437,E2S03338,other,other,women's_tops,NaT,excess2sell,2025-03-03,p3,1.0,9,1.0,20.0,p3,NaN,india,Indonesia,0.0,0.0


# Core

In [ ]:
offers = offers.rename(columns = {'seller_name' : 'seller_short'})

seller_name_to_short_form = {'unilever_indonesia' : 'ulid', 'unilever_malaysia' : 'ulmy', 'unilever_philippines' : 'ulph', 'unilever_thailand' : 'ulth', 'unilever_singapore' : 'ulsg',
                             "l'oreal_malaysia" : "lomy", "l'oreal_philippines" : 'loph', "l'oreal_thailand" : "loth", "l'oreal_india" : "loin", "l'oreal_indonesia" : "loid"}

df_inv['updated_on'] = pd.to_datetime(df_inv['updated_on'])

def apply_seller_short(x):
  if x in seller_name_to_short_form.keys():
    return seller_name_to_short_form[x]
  else:
    return x

df_inv['seller_short'] = df_inv['seller_name'].apply(apply_seller_short)

offers = offers[offers['expiry_date'] != '-']
offers['expiry_date'] = pd.to_datetime(offers['expiry_date'], format = "%d-%b-%Y")
df_inv['expiry_date'] = pd.to_datetime(df_inv['expiry_date'])

In [ ]:
def merge_inv_and_offer(df_inv, df_offer):
  df_inv = pd.merge(df_inv, df_offer[['sku_number', 'expiry_date', 'seller_short', 'date_of_offer', 'actual', 'tracking_no', 'buyer', 'total_units_ordered',
                                      'recovery_rate_percentage', 'offer_price_per_unit_usd', 'total_order_price_usd', 'destination']], 'left', on = ['sku_number', 'expiry_date', 'seller_short'])
  df_inv['offer_day_diff_updated_inv'] = (df_inv['date_of_offer'] - df_inv['updated_on']).dt.days
  df_inv = df_inv[(df_inv.offer_day_diff_updated_inv >= 0) | (df_inv.offer_day_diff_updated_inv.isnull())]
  df_min_offer_day_diff_updated_inv = df_inv.groupby(['sku_number', 'expiry_date', 'seller_short', 'date_of_offer']).agg({'offer_day_diff_updated_inv' : 'min'}).reset_index()\
                                      .rename(columns = {'offer_day_diff_updated_inv' : 'min_offer_day_diff_updated_inv'})
  df_inv = pd.merge(df_inv, df_min_offer_day_diff_updated_inv, 'left', on = ['sku_number', 'expiry_date', 'seller_short', 'date_of_offer'])
  df_inv = df_inv[(df_inv.offer_day_diff_updated_inv == df_inv.min_offer_day_diff_updated_inv) | (df_inv.offer_day_diff_updated_inv.isnull())]
  df_inv.drop('min_offer_day_diff_updated_inv', axis = 1, inplace = True)
  return df_inv

In [ ]:
df_inv = merge_inv_and_offer(df_inv, offers)
df_inv['total_units_ordered'] = df_inv['total_units_ordered'].fillna(0)
# df_inv['sell_thru_rate'] = df_inv['total_units_ordered'] / df_inv['total_number_of_items']
# df_inv['sell_thru_rate'] = np.where(df_inv['sell_thru_rate'] > 1, 1, df_inv['sell_thru_rate'])
df_inv['shelf_life_remaining_days'] = (df_inv['expiry_date'] - df_inv['updated_on']).dt.days

In [ ]:
df_cluster = df_inv[['total_number_of_items', 'seller_name', 'warehouse_country', 'priority', 'retail_price_per_unit_usd', 'asking_price_per_unit_usd',
        'brand', 'product_category', 'product_subcategory']]

categorical_cols = ['priority', 'warehouse_country', 'brand', 'product_category', 'product_subcategory', 'seller_name']
df_encoded = df_cluster.copy()
le_dict = {}

for col in categorical_cols:
    le = LabelEncoder()
    df_encoded[col] = df_encoded[col].astype(str)  # convert NaNs to string "nan"
    df_encoded[col] = le.fit_transform(df_encoded[col])
    le_dict[col] = le

# Step 2: Apply IterativeImputer for missing values
imputer = IterativeImputer(random_state=42)
X_imputed = imputer.fit_transform(df_encoded)

# Step 3: Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)



# Step 4: Run DBSCAN
model = DBSCAN(eps=0.5, min_samples=200)
# model = MeanShift(bin_seeding = True)
df_cluster['cluster'] = model.fit_predict(X_scaled)

labels = df_cluster['cluster'].values
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

if n_clusters >= 2:
    # Remove noise points for silhouette score
    mask = labels != -1
    score = silhouette_score(X_scaled[mask], labels[mask])
    print(f"Silhouette Score (excluding noise): {score:.4f}")
else:
    print("Silhouette Score cannot be computed (requires at least 2 clusters excluding noise).")

Silhouette Score (excluding noise): 0.8465


In [ ]:
df_inv['cluster'] = df_cluster['cluster']
df_inv['cluster'].unique()

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32,
       33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 47, 44, 45, 46, 48, 49,
       50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
       67, 68, 69, 70])

In [ ]:
temp = df_inv.groupby('cluster')['destination'].value_counts().rename('counts').reset_index()
temp = temp.groupby('cluster').destination.first().reset_index()
cluster_to_popular_destination_country = dict(zip(temp.cluster, temp.destination))
DEFAULT = df_inv['destination'].mode().values[0]

In [ ]:
def set_destination(x):
  if pd.isna(x['destination']) and x['cluster'] in cluster_to_popular_destination_country:
    return cluster_to_popular_destination_country[x['cluster']]
  elif pd.isna(x['destination']) and x['cluster'] not in cluster_to_popular_destination_country:
    return DEFAULT
  else:
    return x['destination']

df_inv['destination'] = df_inv.apply(set_destination, axis = 1)
df_inv = df_inv.drop('cluster', axis = 1)

In [ ]:
# dropping duplicates considering all columns except the ids
k = list(set(df_inv.columns) - set([i for i in df_inv.columns if i.endswith('id')]))
df_inv = df_inv.drop_duplicates(subset = k)

In [ ]:
df_inv = df_inv.rename(columns = {'total_number_of_items' : 'total_units'})
df_inv['updated_on'] = pd.to_datetime(df_inv['updated_on'])
df_inv['expiry_date'] = pd.to_datetime(df_inv['expiry_date'])
df_inv = df_inv[df_inv.sku_number != '']
df_inv['shelf_life_remaining_days'] = (df_inv['expiry_date'] - df_inv['updated_on']).dt.days

def remove_percent_sign(x):
  if pd.isna(x):
    return x
  else:
    return x[:-1]
df_inv['recovery_rate_percentage'] = df_inv['recovery_rate_percentage'].apply(remove_percent_sign)
df_inv['recovery_rate_percentage'] = np.where(df_inv['recovery_rate_percentage'] == '', np.nan, df_inv['recovery_rate_percentage'])
df_inv['recovery_rate_percentage'] = df_inv['recovery_rate_percentage'].astype(float)
df_inv['recovery_rate_percentage'] = np.where(df_inv['recovery_rate_percentage'].isnull(), 0 , df_inv['recovery_rate_percentage'])

In [ ]:
key_to_total_units_ordered = df_inv.groupby(['sku_number', 'seller_name', 'expiry_date', 'updated_on']).agg({'total_units_ordered' : 'sum'}).reset_index().rename(columns =
                                                {'total_units_ordered' : 'total_units_ordered_sum'})
df_inv = pd.merge(df_inv, key_to_total_units_ordered, 'left', on = ['sku_number', 'seller_name', 'expiry_date', 'updated_on'])

In [ ]:
temp = df_inv[['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', 'seller_name', 'updated_on', 'priority', 'total_units',
               'shelf_life_remaining_days', 'warehouse_country', 'destination', 'total_units_ordered_sum', 'recovery_rate_percentage']]\
                  .sort_values(by = ['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', 'seller_name', 'updated_on', 'priority', 'warehouse_country', 'destination'])
temp = temp.reset_index(drop = True)

In [ ]:
print(temp.shape)
print(temp.drop_duplicates(subset = ['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', 'seller_name', 'updated_on', 'priority', 'total_units']).shape)

(54892, 14)
(51240, 14)


In [ ]:
def get_prev_inv_occurence_next_to_current(temp):
  new_col_names_for_shifted_inv = dict()
  for i in temp.columns:
    new_col_names_for_shifted_inv[i] = i+'_prev'

  temp_shifted = temp.shift(1)
  temp_shifted = temp_shifted.rename(columns = new_col_names_for_shifted_inv)
  temp = pd.concat([temp, temp_shifted], axis=1)
  return temp

def get_repeated_inv_and_mean_depletion(temp):
  repeated_inv = temp[(temp.sku_number == temp.sku_number_prev) & (temp.expiry_date == temp.expiry_date_prev) & (temp.seller_name == temp.seller_name_prev)]
  repeated_inv['depletion'] = repeated_inv['total_units_prev'] - repeated_inv['total_units']
  repeated_inv['time'] = (repeated_inv['updated_on'] - repeated_inv['updated_on_prev']).dt.days
  repeated_inv = repeated_inv.drop_duplicates(subset = ['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', \
                            'seller_name', 'updated_on', 'priority'])[['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', 'seller_name', 'updated_on_prev',\
                                                                        'priority', 'depletion', 'time', 'total_units_prev', 'total_units', 'priority_prev',\
                                                                        'shelf_life_remaining_days_prev', 'warehouse_country_prev', 'destination_prev',
                                                                       'total_units_ordered_sum_prev', 'recovery_rate_percentage_prev']]
  return repeated_inv

In [ ]:
temp = get_prev_inv_occurence_next_to_current(temp)
repeated_inv = get_repeated_inv_and_mean_depletion(temp)

# since current was match to previous, hence correcting the current row previous priority order to previous one
repeated_inv['priority'] = np.where(repeated_inv.priority != repeated_inv.priority_prev, repeated_inv.priority_prev, repeated_inv.priority)

In [ ]:
print(repeated_inv[repeated_inv.time > 90].shape)
print('Removing above number of rows as sales cycle is fixed to 90 days max for any cat or sub cat after which it is assumed to finish')
repeated_inv = repeated_inv[repeated_inv.time <= 90]

(2899, 18)
Removing above number of rows as sales cycle is fixed to 90 days max for any cat or sub cat after which it is assumed to finish


In [ ]:
mean_depletion_per_cat_core_inv = repeated_inv.groupby(['product_category']).depletion.mean().reset_index().sort_values(by = 'depletion')

In [ ]:
low_depletion_cats = list(mean_depletion_per_cat_core_inv[mean_depletion_per_cat_core_inv['depletion'] <= mean_depletion_per_cat_core_inv.depletion.quantile(0.33)].product_category)
mid_depletion_cats = list(mean_depletion_per_cat_core_inv[mean_depletion_per_cat_core_inv['depletion'] <= mean_depletion_per_cat_core_inv.depletion.quantile(0.67)].product_category)
mid_depletion_cats = [i for i in mid_depletion_cats if i not in low_depletion_cats]
high_depletion_cats = list(mean_depletion_per_cat_core_inv[mean_depletion_per_cat_core_inv['depletion'] <= mean_depletion_per_cat_core_inv.depletion.quantile(1)].product_category)
high_depletion_cats = [i for i in high_depletion_cats if i not in low_depletion_cats + mid_depletion_cats]

In [ ]:
print(low_depletion_cats, '\n', mid_depletion_cats, '\n', high_depletion_cats)

['beverages', 'fashion_-_accessories', 'home', 'oral_care'] 
 ['makeup', 'fragrance', 'skin_care'] 
 ['baby', 'hair_care', 'home_care', 'food']


In [ ]:
# do disposal correction

In [ ]:
def set_depletion(row):
  if row['depletion'] < 0:
    return row['total_units_prev']
  elif row['product_category'] in high_depletion_cats and row['time'] <= 20:
    return row['depletion']
  elif row['product_category'] in high_depletion_cats and row['time'] > 20:
    return row['total_units_prev']
  elif row['product_category'] in mid_depletion_cats and row['time'] <= 45:
    return row['depletion']
  elif row['product_category'] in mid_depletion_cats and row['time'] > 45:
    return row['total_units_prev']
  elif row['product_category'] in low_depletion_cats and row['time'] <= 60:
    return row['depletion']
  elif row['product_category'] in low_depletion_cats and row['time'] > 60:
    return row['total_units_prev']

In [ ]:
repeated_inv['depletion'] = repeated_inv.apply(set_depletion, axis=1)
repeated_inv = repeated_inv.reset_index(drop = True)

In [ ]:
repeated_inv = repeated_inv.rename(columns = {'total_units_ordered_sum_prev' : 'pollen_liquidated_units', 'recovery_rate_percentage_prev' : 'pollen_recovery_rate_percentage'})

In [ ]:
COLUMNS_FOR_DEPLETION_MODELING = ['sku_number', 'brand', 'product_category', 'product_subcategory', 'seller_name', 'time']

In [ ]:
repeated_inv = repeated_inv[COLUMNS_FOR_DEPLETION_MODELING + ['updated_on_prev', 'priority', 'total_units_prev', 'depletion', 'shelf_life_remaining_days_prev', 'warehouse_country_prev',
                                                              'destination_prev', 'pollen_liquidated_units', 'pollen_recovery_rate_percentage']]
repeated_inv_non_core = repeated_inv_non_core[COLUMNS_FOR_DEPLETION_MODELING + ['updated_on_prev', 'priority', 'total_units_prev', 'depletion', 'shelf_life_remaining_days_prev',
                                                                                'warehouse_country_prev', 'destination_prev', 'pollen_liquidated_units', 'pollen_recovery_rate_percentage']]
print(repeated_inv.shape, repeated_inv_non_core.shape)

(19109, 15) (2237, 15)


In [ ]:
repeated_inv = pd.concat([repeated_inv, repeated_inv_non_core], ignore_index=True)
repeated_inv['destination_prev'] = repeated_inv['destination_prev'].str.lower().str.replace(' ', '_')
repeated_inv.depletion['depletion'] = np.where(repeated_inv.depletion < repeated_inv.pollen_liquidated_units, repeated_inv.pollen_liquidated_units, repeated_inv.depletion)
repeated_inv.shape

(21346, 15)

In [ ]:
repeated_inv['destination_prev'].unique()

array(['indonesia', 'afghanistan', 'uae', 'singapore', 'bangladesh',
       'pakistan', 'iran', 'malaysia', 'africa', 'india', 'hong_kong',
       'new_zealand', 'myanmar', 'philippines', 'czech_republic',
       'thailand'], dtype=object)

In [ ]:
repeated_inv.sample(2)

,sku_number,brand,product_category,product_subcategory,seller_name,time,updated_on_prev,priority,total_units_prev,depletion,shelf_life_remaining_days_prev,warehouse_country_prev,destination_prev,pollen_liquidated_units,pollen_recovery_rate_percentage
20004,E2S02860,home,home_-_other,women's_tops,excess2sell,9,2025-03-03,p3,1.0,1.0,NaN,india,indonesia,0.0,0.0
19238,3CEID0018,3_ce,makeup,eye_makeup,eblo,72,2023-08-14,p1,5.0,5.0,371.0,None,indonesia,0.0,0.0


In [ ]:
def correct_p1_depletion(df):
  if df['priority'] == 'p1' and np.random.uniform(low=1, high=99, size=(1,))[0] <= 80:
    return np.random.uniform(low=5, high=20, size=(1,))[0]/ 100 * df['depletion']
  else:
    return df['depletion']

repeated_inv['depletion'] = repeated_inv.apply(correct_p1_depletion, axis = 1)
repeated_inv['pollen_liquidated_units'] = np.where(repeated_inv.pollen_liquidated_units > repeated_inv.depletion, repeated_inv.depletion, repeated_inv.pollen_liquidated_units)
repeated_inv = repeated_inv.reset_index(drop = True)

In [ ]:
high_time_dummy = repeated_inv.copy()

def get_dummy_high_time(df):
    if pd.isna(df['time']):
      return df['time']
    else:
      return np.random.uniform(
            low=df['time'],
            high=1.5 * df['time'], size=(1,))[0]

high_time_dummy['time'] = high_time_dummy.apply(get_dummy_high_time, axis = 1)
high_time_dummy['orig_time'] = repeated_inv['time']
high_time_dummy['time_factor'] = high_time_dummy['time'] / high_time_dummy['orig_time']
high_time_dummy['depletion'] = high_time_dummy['depletion'] * high_time_dummy['time_factor']
high_time_dummy['total_units_prev'] = np.where(high_time_dummy['total_units_prev'] < high_time_dummy['depletion'],
                                                    high_time_dummy['depletion'], high_time_dummy['total_units_prev'])

In [ ]:
low_time_dummy = repeated_inv.copy()

def get_dummy_low_time(df):
    if pd.isna(df['time']):
      return df['time']
    else:
      return np.random.uniform(
            low=0.5 * df['time'],
            high=df['time'], size=(1,))[0]

low_time_dummy['time'] = low_time_dummy.apply(get_dummy_low_time, axis = 1)
low_time_dummy['orig_time'] = repeated_inv['time']
low_time_dummy['time_factor'] = low_time_dummy['time'] / low_time_dummy['orig_time']
low_time_dummy['depletion'] = low_time_dummy['depletion'] * low_time_dummy['time_factor']
low_time_dummy['total_units_prev'] = np.where(low_time_dummy['total_units_prev'] < low_time_dummy['depletion'],
                                                    low_time_dummy['depletion'], low_time_dummy['total_units_prev'])

In [ ]:
high_shelf_life_dummy = repeated_inv.copy()

def get_dummy_high_shelf_life(df):
    if pd.isna(df['shelf_life_remaining_days_prev']):
      return df['shelf_life_remaining_days_prev']
    else:
      return np.random.uniform(
            low=df['shelf_life_remaining_days_prev'],
            high=1.5 * df['shelf_life_remaining_days_prev'], size=(1,))[0]

high_shelf_life_dummy['shelf_life_remaining_days_prev'] = high_shelf_life_dummy.apply(get_dummy_high_shelf_life, axis = 1)
high_shelf_life_dummy['orig_shelf_life'] = repeated_inv['shelf_life_remaining_days_prev']
high_shelf_life_dummy['shelf_life_factor'] = high_shelf_life_dummy['shelf_life_remaining_days_prev'] / high_shelf_life_dummy['orig_shelf_life']
high_shelf_life_dummy['depletion'] = high_shelf_life_dummy['depletion'] * high_shelf_life_dummy['shelf_life_factor']
high_shelf_life_dummy['total_units_prev'] = np.where(high_shelf_life_dummy['total_units_prev'] < high_shelf_life_dummy['depletion'],
                                                    high_shelf_life_dummy['depletion'], high_shelf_life_dummy['total_units_prev'])

In [ ]:
low_shelf_life_dummy = repeated_inv.copy()

def get_dummy_low_shelf_life(df):
    if pd.isna(df['shelf_life_remaining_days_prev']):
      return df['shelf_life_remaining_days_prev']
    else:
      return np.random.uniform(
            low=0.5 * df['shelf_life_remaining_days_prev'],
            high=df['shelf_life_remaining_days_prev'], size=(1,))[0]

low_shelf_life_dummy['shelf_life_remaining_days_prev'] = low_shelf_life_dummy.apply(get_dummy_low_shelf_life, axis = 1)
low_shelf_life_dummy['orig_shelf_life'] = repeated_inv['shelf_life_remaining_days_prev']
low_shelf_life_dummy['shelf_life_factor'] = low_shelf_life_dummy['shelf_life_remaining_days_prev'] / low_shelf_life_dummy['orig_shelf_life']
low_shelf_life_dummy['depletion'] = low_shelf_life_dummy['depletion'] * low_shelf_life_dummy['shelf_life_factor']
low_shelf_life_dummy['total_units_prev'] = np.where(low_shelf_life_dummy['total_units_prev'] < low_shelf_life_dummy['depletion'],
                                                    low_shelf_life_dummy['depletion'], low_shelf_life_dummy['total_units_prev'])

In [ ]:
repeated_inv = pd.concat([repeated_inv, high_shelf_life_dummy, low_shelf_life_dummy, low_time_dummy, high_time_dummy], ignore_index=True)
repeated_inv = repeated_inv.drop_duplicates()
repeated_inv = repeated_inv.reset_index(drop = True)
repeated_inv = repeated_inv.drop(['orig_shelf_life', 'shelf_life_factor', 'orig_time', 'time_factor'], axis = 1)

In [ ]:
# some duplicates because of nan shelf life

print(repeated_inv.duplicated().sum())
repeated_inv = repeated_inv.drop_duplicates()

20


## Listing type

In [ ]:
def get_listing_type(df):
  if df['shelf_life_remaining_days_prev'] > 365*2:
    return 'fresh'
  elif df['shelf_life_remaining_days_prev'] > 365 and df['pollen_recovery_rate_percentage'] > 0 and df['pollen_recovery_rate_percentage'] < 10:
    return 'obsolete'
  else:
    return 'excess'

repeated_inv['listing_condition'] = repeated_inv.apply(get_listing_type, axis = 1)

number_of_rows_to_randomize = int(repeated_inv.shape[0]*0.05)

change = repeated_inv.sample(number_of_rows_to_randomize//2).index
repeated_inv.loc[change,'listing_condition'] = 'damaged'
change = repeated_inv.sample(number_of_rows_to_randomize//2).index
repeated_inv.loc[change,'listing_condition'] = 'made_to_order'

In [ ]:
repeated_inv.listing_condition.value_counts()

,count
listing_condition,
excess,90531
fresh,5633
made_to_order,2536
damaged,2490
obsolete,253


## Liquidation channel

In [ ]:
repeated_inv['depletion'] = np.where(repeated_inv['depletion'] < repeated_inv['pollen_liquidated_units'], repeated_inv['pollen_liquidated_units'], repeated_inv['depletion'])
repeated_inv['depletion_other_than_pollen'] = repeated_inv['depletion'] - repeated_inv['pollen_liquidated_units']
repeated_inv['liquidation_channel'] = [['friends/family', 'distributor', 'retailer', 'liquidator', 'disposal', 'pollen']]*len(repeated_inv)
repeated_inv = repeated_inv.explode('liquidation_channel').reset_index(drop = True)

In [ ]:
def get_depleted_units_via_liquidation_channel(df):
  if df['priority'] == 'p1':
    if df['liquidation_channel'] == 'friends/family':
      return 0
    elif df['liquidation_channel'] == 'distributor':
      return 0.2 * 0.1 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'retailer':
      return 0.2 * 0.1 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'liquidator':
      return 0.2 * 0.8 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'disposal':
      return 0.8 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'pollen':
      return df['pollen_liquidated_units']

  if df['priority'] == 'p2':
    if df['liquidation_channel'] == 'friends/family':
      return 0.6 * 0.05 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'distributor':
      return 0.6 * 0.25 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'retailer':
      return 0.6 * 0.20 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'liquidator':
      return 0.6 * 0.5 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'disposal':
      return 0.4 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'pollen':
      return df['pollen_liquidated_units']

  if df['priority'] == 'p3':
    if df['liquidation_channel'] == 'friends/family':
      return 0.8 * 0.1 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'distributor':
      return 0.8 * 0.3 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'retailer':
      return 0.8 * 0.3 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'liquidator':
      return 0.8 * 0.3 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'disposal':
      return 0.2 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'pollen':
      return df['pollen_liquidated_units']

In [ ]:
repeated_inv['final_depletion'] = repeated_inv.apply(get_depleted_units_via_liquidation_channel, axis = 1)
repeated_inv['depletion'] = repeated_inv['final_depletion']
repeated_inv = repeated_inv.drop(['pollen_liquidated_units', 'depletion_other_than_pollen', 'final_depletion', 'pollen_recovery_rate_percentage'], axis = 1)

In [ ]:
# after above explode, the relationship between time and liquidation channel + time and depletion number + time and shelf_life_remaining_days_prev
# went for a toss. This is for the correction to keep the sensitivity intact

repeated_inv['channel_group'] = repeated_inv.index // len(['friends/family', 'distributor', 'retailer', 'liquidator', 'disposal', 'pollen'])
max_depletion_in_each_group = repeated_inv.groupby('channel_group').agg({'depletion' : 'max'}).reset_index().rename(columns = {'depletion' : 'max_depletion'})
repeated_inv = pd.merge(repeated_inv, max_depletion_in_each_group, 'left', on = 'channel_group')

In [ ]:
def time_correction(df):
  if df['max_depletion'] == 0:
    return df['time']
  else:
    return df['time'] * df['depletion'] / df['max_depletion']

repeated_inv['time'] = repeated_inv.apply(time_correction, axis = 1)

def shelf_life_correction(df):
  if df['max_depletion'] == 0:
    return df['shelf_life_remaining_days_prev']
  else:
    return df['shelf_life_remaining_days_prev'] * df['depletion'] / df['max_depletion']

repeated_inv['shelf_life_remaining_days_prev'] = repeated_inv.apply(shelf_life_correction, axis = 1)
repeated_inv = repeated_inv.drop(['channel_group', 'max_depletion'], axis = 1)

In [ ]:
sellers['seller_name'] = sellers['seller_name'].str.lower().str.replace(' ', '_')
sellers['persona_seller_type'] = sellers['persona_seller_type'].str.lower().str.replace(' ', '_')
repeated_inv = pd.merge(repeated_inv, sellers, 'left', on = 'seller_name')
print(repeated_inv.priority.isnull().sum(), repeated_inv.persona_seller_type.isnull().sum())

18492 144


In [ ]:
repeated_inv['priority'] = np.where(repeated_inv.priority.isnull(), '', repeated_inv.priority)
repeated_inv['persona_seller_type'] = np.where(repeated_inv.persona_seller_type.isnull(), '', repeated_inv.persona_seller_type)

repeated_inv = repeated_inv[~(repeated_inv['total_units_prev'] == 0)]
repeated_inv['depletion_percent'] = repeated_inv['depletion'] / repeated_inv['total_units_prev']
repeated_inv = repeated_inv[~repeated_inv['depletion_percent'].isnull()]
repeated_inv['depletion_percent'] = repeated_inv['depletion_percent']*100

In [ ]:
repeated_inv = repeated_inv.rename(columns = {'warehouse_country_prev' : 'warehouse_country', 'shelf_life_remaining_days_prev' : 'shelf_life_remaining_days',
                                              'total_units_prev' : 'total_units', 'updated_on_prev' : 'updated_on', 'destination_prev' : 'destination'})

In [ ]:
repeated_inv.shape

(565458, 17)

In [ ]:
print(repeated_inv.shape)
print(repeated_inv[repeated_inv.time < 0].shape)
repeated_inv['time'] = np.where(repeated_inv.time < 0, -repeated_inv.time, repeated_inv.time)

(565458, 17)
(1450, 17)


In [ ]:
priority_to_shelf_life_median = repeated_inv.groupby('priority').shelf_life_remaining_days.median().reset_index()
priority_to_shelf_life_median = dict(zip(list(priority_to_shelf_life_median.priority), list(priority_to_shelf_life_median.shelf_life_remaining_days)))
priority_to_shelf_life_median

{'p1': 66.94190104745843, 'p2': 179.48733180871426, 'p3': 439.0}

In [ ]:
def fill_na_shelf_life_remaining_days(df):
  if pd.isna(df['shelf_life_remaining_days']) and df['priority'] == 'p1':
    return priority_to_shelf_life_median['p1']
  elif pd.isna(df['shelf_life_remaining_days']) and df['priority'] == 'p2':
    return priority_to_shelf_life_median['p2']
  elif pd.isna(df['shelf_life_remaining_days']) and df['priority'] == 'p3':
    return priority_to_shelf_life_median['p3']
  else:
    return df['shelf_life_remaining_days']

repeated_inv['shelf_life_remaining_days'] = repeated_inv.apply(fill_na_shelf_life_remaining_days, axis = 1)
repeated_inv['product_subcategory'] = np.where(repeated_inv.product_subcategory == 'frozen_food', 'frozen_foods', repeated_inv.product_subcategory)

In [ ]:
repeated_inv.sample(2)

,sku_number,brand,product_category,product_subcategory,seller_name,time,updated_on,priority,total_units,depletion,shelf_life_remaining_days,warehouse_country,destination,listing_condition,liquidation_channel,persona_seller_type,depletion_percent
143244,67843668,pepsodent,oral_care,tooth_brush,unilever_indonesia,24.000000,2024-09-09,p2,48.0,0.0,256.139354,indonesia,indonesia,excess,friends/family,principal,0.0
580097,G4075800,maybelline,makeup,face_makeup,l'oreal_philippines,22.359233,2022-01-21,p3,3000.0,0.0,558.000000,philippines,indonesia,excess,pollen,principal,0.0


In [ ]:
def get_week_number(x):
  if pd.isna(x):
    return np.nan
  elif x.isocalendar().week / 10 < 1:
    return '0' + str(x.isocalendar().week)
  else:
    return str(x.isocalendar().week)

def get_year(x):
  if pd.isna(x):
    return np.nan
  else:
    return str(x.year)

def get_month(x):
  if pd.isna(x):
    return np.nan
  elif x.month / 10 < 1:
    return '0' + str(x.month)
  else:
    return str(x.month)

repeated_inv['week_number'] = repeated_inv['updated_on'].apply(get_week_number)
repeated_inv['month'] = repeated_inv['updated_on'].apply(get_month)
repeated_inv['year'] = repeated_inv['updated_on'].apply(get_year)
repeated_inv['year_plus_week_number'] = repeated_inv['year'] + '_' + repeated_inv['week_number']
repeated_inv['year_plus_month'] = repeated_inv['year'] + '_' + repeated_inv['month']

In [ ]:
repeated_inv.shape

(565458, 22)

# PSI Chart

# impute demand where demand is zero

In [ ]:
category_total_shelf_life = category_total_shelf_life.rename(columns = {'Product Category' : 'product_category', 'Product Sub Category' : 'product_subcategory',
                                              'Est. Product Shelf Life months' : 'total_shelf_life_months'})
category_total_shelf_life = category_total_shelf_life.drop(['Proposed Buyer Product Preferences'], axis = 1)

category_total_shelf_life['product_category'] = category_total_shelf_life['product_category'].str.lower().str.replace(' ', '_')
category_total_shelf_life['product_subcategory'] = category_total_shelf_life['product_subcategory'].str.lower().str.replace(' ', '_')

def get_months(x):
  return int(x.split()[0])

category_total_shelf_life['total_shelf_life_months'] = category_total_shelf_life['total_shelf_life_months'].apply(get_months)
repeated_inv = pd.merge(repeated_inv, category_total_shelf_life[['product_category', 'product_subcategory', 'total_shelf_life_months']],
                        'left', on = ['product_category', 'product_subcategory'])
print(repeated_inv['total_shelf_life_months'].isnull().sum())

22876


In [ ]:
repeated_inv['total_shelf_life_months'] = repeated_inv['total_shelf_life_months'] * 30
repeated_inv = repeated_inv.rename(columns = {'total_shelf_life_months' : 'total_shelf_life_days'})
repeated_inv['percent_shelf_life_remaining'] = repeated_inv['shelf_life_remaining_days'] / repeated_inv['total_shelf_life_days']*100

def get_priority(x):
  if x <= 25:
    return 'p1'
  elif 25 < x <= 75:
    return 'p2'
  else:
    return 'p3'

repeated_inv['priority'] = repeated_inv['percent_shelf_life_remaining'].apply(get_priority)

# PSI Cat week

In [ ]:
df_psi_cat_week = repeated_inv.groupby(['product_category', 'priority', 'warehouse_country', 'destination', 'year_plus_week_number']).agg({'total_units' : 'sum',
                                                                                                                   'depletion' : 'sum'}).reset_index()
df_psi_cat_week['demand_score'] = df_psi_cat_week['depletion'] / df_psi_cat_week['total_units']*100
print(df_psi_cat_week.shape)
df_psi_cat_week.sample()

(3000, 8)


,product_category,priority,warehouse_country,destination,year_plus_week_number,total_units,depletion,demand_score
2063,makeup,p3,thailand,thailand,2024_47,15943.33862,81.727883,0.512615


In [ ]:
def fill_missing_weeks_to_present(df):
    # Convert 'year_plus_week_number' into a format that allows for comparison
    def convert_to_week_num(week_str):
        year, week = map(int, week_str.split('_'))
        return year * 52 + week  # Unique number for each week

    # Sort DataFrame by 'year_plus_week_number'
    df = df.sort_values(by='year_plus_week_number').reset_index(drop=True)

    # Create a new list for the filled data
    filled_data = []

    for i in range(len(df) - 1):
        filled_data.append(df.iloc[i])  # Add current row

        # Identify missing weeks
        current_week = convert_to_week_num(df.iloc[i]['year_plus_week_number'])
        next_week = convert_to_week_num(df.iloc[i + 1]['year_plus_week_number'])

        # Add missing weeks
        for missing_week in range(current_week + 1, next_week):
            year = missing_week // 52
            week = missing_week % 52 if missing_week % 52 != 0 else 52
            filled_row = df.iloc[i].copy()
            filled_row['year_plus_week_number'] = f"{year}_{week:02d}"
            filled_data.append(filled_row)

    # Append the last row
    filled_data.append(df.iloc[-1])

    # Fill till the current week
    today = datetime.today()
    current_year, current_week = today.isocalendar()[:2]

    last_week = convert_to_week_num(df.iloc[-1]['year_plus_week_number'])
    current_week_num = current_year * 52 + current_week

    for missing_week in range(last_week + 1, current_week_num + 1):
        year = missing_week // 52
        week = missing_week % 52 if missing_week % 52 != 0 else 52
        filled_row = df.iloc[-1].copy()
        filled_row['year_plus_week_number'] = f"{year}_{week:02d}"
        filled_data.append(filled_row)

    # Create a new DataFrame
    filled_df = pd.DataFrame(filled_data)
    return filled_df.reset_index(drop=True)


def fill_demand_score_with_previous_and_next_non_null_values(df):
    # Ensure demand_score is float for random values
    df['demand_score'] = df['demand_score'].astype(float)

    # Store the original non-null demand_score values to track true non-null values
    original_non_null_values = df['demand_score'].copy()

    # Iterate through rows to handle NaN values
    for i in range(len(df)):
        if pd.isna(df.loc[i, 'demand_score']):
            # Find previous non-null value from original data (not the filled value)
            prev_value = original_non_null_values[:i].dropna().iloc[-1] if i > 0 else 0

            # Find next non-null value
            next_value_series = df.loc[i+1:, 'demand_score'].dropna()
            next_value = next_value_series.iloc[0] if not next_value_series.empty else None

            # Fill logic
            if next_value is not None:
                df.loc[i, 'demand_score'] = random.uniform(prev_value, next_value)
            else:
                df.loc[i, 'demand_score'] = random.uniform(0, prev_value)

    return df

In [ ]:
final_psi_cat_week_df = pd.DataFrame()
filter_vals = df_psi_cat_week[['product_category', 'priority', 'warehouse_country', 'destination']].drop_duplicates().values
print(len(filter_vals))

for cat, p, s, d in filter_vals:
  temp = df_psi_cat_week[(df_psi_cat_week['product_category'] == cat) & (df_psi_cat_week['priority'] == p) & (df_psi_cat_week['warehouse_country'] == s)
                      & (df_psi_cat_week['destination'] == d)].sort_values(by = 'year_plus_week_number')

  new_temp = fill_missing_weeks_to_present(temp)

  new_temp = pd.merge(new_temp[['product_category', 'priority', 'warehouse_country', 'destination', 'year_plus_week_number']],
         temp, 'left', on = ['product_category', 'priority', 'warehouse_country', 'destination', 'year_plus_week_number'])
  fill_demand_score_with_previous_and_next_non_null_values(new_temp)
  final_psi_cat_week_df = pd.concat([final_psi_cat_week_df, new_temp], ignore_index=True)

378


In [ ]:
plot_temp = final_psi_cat_week_df[(final_psi_cat_week_df['product_category'] == 'skin_care') & (final_psi_cat_week_df['priority'] == 'p3') &
                                (final_psi_cat_week_df['warehouse_country'] == 'thailand') & (final_psi_cat_week_df['destination'] == 'pakistan')]
plot_temp = plot_temp[-32:] # last 8 months (32 weeks) plot
fig = px.line(plot_temp, x="year_plus_week_number", y="demand_score", title='PSI Cat Week')
fig.show()

In [ ]:
demand_psi_cat_week_country_maps = final_psi_cat_week_df.groupby(['product_category', 'priority', 'warehouse_country', 'destination']).agg({'demand_score' : 'nunique'})\
.reset_index().rename(columns = {'demand_score' : 'unique_demand_scores'})
print(len(demand_psi_cat_week_country_maps))
demand_psi_cat_week_country_maps = demand_psi_cat_week_country_maps[demand_psi_cat_week_country_maps.unique_demand_scores >= 32] # 8 months

demand_psi_cat_week_country_maps = demand_psi_cat_week_country_maps.drop('unique_demand_scores', axis = 1)
print(len(demand_psi_cat_week_country_maps))

378
269


# PSI Cat month

In [ ]:
df_psi_cat_month = repeated_inv.groupby(['product_category', 'priority', 'warehouse_country', 'destination', 'year_plus_month']).agg({'total_units' : 'sum',
                                                                                                                   'depletion' : 'sum'}).reset_index()
df_psi_cat_month['demand_score'] = df_psi_cat_month['depletion'] / df_psi_cat_month['total_units']*100
print(df_psi_cat_month.shape)
df_psi_cat_month.sample()

(2509, 8)


,product_category,priority,warehouse_country,destination,year_plus_month,total_units,depletion,demand_score
32,baby,p2,indonesia,indonesia,2022_12,5760.0,0.0,0.0


In [ ]:
def fill_missing_year_plus_month_for_cat(df):
    # Ensure 'year_plus_month' is string for easy manipulation
    df['year_plus_month'] = df['year_plus_month'].astype(str)

    # Identify min and max 'year_plus_month' in the original data
    min_year_month = df['year_plus_month'].min()
    max_year_month = datetime.today().strftime('%Y_%m')  # Last month from today

    # Create continuous 'year_plus_month' range
    date_range = pd.date_range(start=min_year_month.replace('_', '-'),
                               end=max_year_month.replace('_', '-'),
                               freq='MS').strftime('%Y_%m').tolist()

    # Iterate through unique combinations of keys
    filled_dfs = []
    for _, group in df.groupby(['product_category', 'priority', 'warehouse_country', 'destination']):
        product_details = group.iloc[0][['product_category', 'priority', 'warehouse_country', 'destination']].to_dict()

        # Create a DataFrame for the full year_plus_month range
        full_range_df = pd.DataFrame({'year_plus_month': date_range})

        # Merge with the original group
        merged_df = full_range_df.merge(group, on='year_plus_month', how='left')

        # Fill forward missing data for non-date columns
        for col in ['product_category', 'priority', 'warehouse_country', 'destination',
                    'total_units', 'depletion']:
            merged_df[col].fillna(method='ffill', inplace=True)

        # Fill `demand_score` with appropriate logic
        original_demand_score = group[['year_plus_month', 'demand_score']].set_index('year_plus_month')['demand_score']

        # Impute demand_score using random values in the desired range
        def impute_demand_score(row):
            if pd.notna(row['demand_score']):
                return row['demand_score']

            # Identify previous and next non-null values in the original data
            prev_value_candidates = original_demand_score.loc[
                original_demand_score.index < row['year_plus_month']
            ]
            prev_value = prev_value_candidates.iloc[-1] if not prev_value_candidates.empty else 0  # Safe fallback to zero

            next_value_candidates = original_demand_score.loc[
                original_demand_score.index > row['year_plus_month']
            ]
            next_value = next_value_candidates.iloc[0] if not next_value_candidates.empty else None

            # Impute demand score accordingly
            if next_value is not None:
                return np.random.uniform(prev_value, next_value)
            else:
                return np.random.uniform(0, prev_value)

        merged_df['demand_score'] = merged_df.apply(impute_demand_score, axis=1)

        filled_dfs.append(merged_df)

    # Combine all filled DataFrames
    filled_df = pd.concat(filled_dfs).reset_index(drop=True)

    return filled_df

In [ ]:
final_psi_cat_month_df = fill_missing_year_plus_month_for_cat(df_psi_cat_month)

In [ ]:
plot_temp = final_psi_cat_month_df[(final_psi_cat_month_df['product_category'] == 'hair_care') & (final_psi_cat_month_df['priority'] == 'p3') &
                                (final_psi_cat_month_df['warehouse_country'] == 'malaysia') & (final_psi_cat_month_df['destination'] == 'malaysia')]
plot_temp = plot_temp[-8:] # last 8 months plot
fig = px.line(plot_temp, x="year_plus_month", y="demand_score", title='PSI Cat Month')
fig.show()

In [ ]:
demand_psi_cat_month_country_maps = final_psi_cat_month_df.groupby(['product_category', 'priority', 'warehouse_country', 'destination']).agg({'demand_score' : 'nunique'})\
.reset_index().rename(columns = {'demand_score' : 'unique_demand_scores'})
print(len(demand_psi_cat_month_country_maps))
demand_psi_cat_month_country_maps = demand_psi_cat_month_country_maps[demand_psi_cat_month_country_maps.unique_demand_scores >= 8] # 8 months

demand_psi_cat_month_country_maps = demand_psi_cat_month_country_maps.drop('unique_demand_scores', axis = 1)
print(len(demand_psi_cat_month_country_maps))

378
271


# PSI Sub cat week

In [ ]:
df_psi_subcat_week = repeated_inv.groupby(['product_subcategory', 'priority', 'warehouse_country', 'destination', 'year_plus_week_number']).agg({'total_units' : 'sum',
                                                                                                                   'depletion' : 'sum'}).reset_index()
df_psi_subcat_week['demand_score'] = df_psi_subcat_week['depletion'] / df_psi_subcat_week['total_units']*100
print(df_psi_subcat_week.shape)
df_psi_subcat_week.sample()

(5977, 8)


,product_subcategory,priority,warehouse_country,destination,year_plus_week_number,total_units,depletion,demand_score
3418,health_drinks,p3,indonesia,indonesia,2024_19,2743.923263,1066.211624,38.857195


In [ ]:
final_psi_subcat_week_df = pd.DataFrame()
filter_vals = df_psi_subcat_week[['product_subcategory', 'priority', 'warehouse_country', 'destination']].drop_duplicates().values
print(len(filter_vals))

for subcat, p, s, d in filter_vals:
  temp = df_psi_subcat_week[(df_psi_subcat_week['product_subcategory'] == subcat) & (df_psi_subcat_week['priority'] == p) & (df_psi_subcat_week['warehouse_country'] == s) &
                            (df_psi_subcat_week['destination'] == d)].sort_values(by = 'year_plus_week_number')

  new_temp = fill_missing_weeks_to_present(temp)

  new_temp = pd.merge(new_temp[['product_subcategory', 'priority', 'warehouse_country', 'destination', 'year_plus_week_number']],
         temp, 'left', on = ['product_subcategory', 'priority', 'warehouse_country', 'destination', 'year_plus_week_number'])
  fill_demand_score_with_previous_and_next_non_null_values(new_temp)
  final_psi_subcat_week_df = pd.concat([final_psi_subcat_week_df, new_temp], ignore_index=True)

880


In [ ]:
plot_temp = final_psi_subcat_week_df[(final_psi_subcat_week_df['product_subcategory'] == 'conditioner') & (final_psi_subcat_week_df['priority'] == 'p1') &
                                (final_psi_subcat_week_df['warehouse_country'] == 'malaysia') & (final_psi_subcat_week_df['destination'] == 'malaysia')]
plot_temp = plot_temp[-32:] # last 8 months (32 weeks) plot
fig = px.line(plot_temp, x="year_plus_week_number", y="demand_score", title='PSI SubCat Week')
fig.show()

In [ ]:
demand_psi_subcat_week_country_maps = final_psi_subcat_week_df.groupby(['product_subcategory', 'priority', 'warehouse_country', 'destination']).agg({'demand_score' : 'nunique'})\
.reset_index().rename(columns = {'demand_score' : 'unique_demand_scores'})
print(len(demand_psi_subcat_week_country_maps))
demand_psi_subcat_week_country_maps = demand_psi_subcat_week_country_maps[demand_psi_subcat_week_country_maps.unique_demand_scores >= 32] # 8 months

demand_psi_subcat_week_country_maps = demand_psi_subcat_week_country_maps.drop('unique_demand_scores', axis = 1)
print(len(demand_psi_subcat_week_country_maps))

880
629


# PSI Sub cat month

In [ ]:
df_psi_subcat_month = repeated_inv.groupby(['product_subcategory', 'priority', 'warehouse_country', 'destination', 'year_plus_month']).agg({'total_units' : 'sum',
                                                                                                                   'depletion' : 'sum'}).reset_index()
df_psi_subcat_month['demand_score'] = df_psi_subcat_month['depletion'] / df_psi_subcat_month['total_units']*100
print(df_psi_subcat_month.shape)
df_psi_subcat_month.sample()

(5181, 8)


,product_subcategory,priority,warehouse_country,destination,year_plus_month,total_units,depletion,demand_score
2027,face_makeup,p1,malaysia,malaysia,2023_12,2013.145451,133.655944,6.63916


In [ ]:
def fill_missing_year_plus_month_for_subcat(df):
    # Ensure 'year_plus_month' is string for easy manipulation
    df['year_plus_month'] = df['year_plus_month'].astype(str)

    # Identify min and max 'year_plus_month' in the original data
    min_year_month = df['year_plus_month'].min()
    max_year_month = datetime.today().strftime('%Y_%m')  # Last month from today

    # Create continuous 'year_plus_month' range
    date_range = pd.date_range(start=min_year_month.replace('_', '-'),
                               end=max_year_month.replace('_', '-'),
                               freq='MS').strftime('%Y_%m').tolist()

    # Iterate through unique combinations of keys
    filled_dfs = []
    for _, group in df.groupby(['product_subcategory', 'priority', 'warehouse_country', 'destination']):
        product_details = group.iloc[0][['product_subcategory', 'priority', 'warehouse_country', 'destination']].to_dict()

        # Create a DataFrame for the full year_plus_month range
        full_range_df = pd.DataFrame({'year_plus_month': date_range})

        # Merge with the original group
        merged_df = full_range_df.merge(group, on='year_plus_month', how='left')

        # Fill forward missing data for non-date columns
        for col in ['product_subcategory', 'priority', 'warehouse_country', 'destination',
                    'total_units', 'depletion']:
            merged_df[col].fillna(method='ffill', inplace=True)

        # Fill `demand_score` with appropriate logic
        original_demand_score = group[['year_plus_month', 'demand_score']].set_index('year_plus_month')['demand_score']

        # Impute demand_score using random values in the desired range
        def impute_demand_score(row):
            if pd.notna(row['demand_score']):
                return row['demand_score']

            # Identify previous and next non-null values in the original data
            prev_value_candidates = original_demand_score.loc[
                original_demand_score.index < row['year_plus_month']
            ]
            prev_value = prev_value_candidates.iloc[-1] if not prev_value_candidates.empty else 0  # Safe fallback to zero

            next_value_candidates = original_demand_score.loc[
                original_demand_score.index > row['year_plus_month']
            ]
            next_value = next_value_candidates.iloc[0] if not next_value_candidates.empty else None

            # Impute demand score accordingly
            if next_value is not None:
                return np.random.uniform(prev_value, next_value)
            else:
                return np.random.uniform(0, prev_value)

        merged_df['demand_score'] = merged_df.apply(impute_demand_score, axis=1)

        filled_dfs.append(merged_df)

    # Combine all filled DataFrames
    filled_df = pd.concat(filled_dfs).reset_index(drop=True)

    return filled_df

In [ ]:
final_psi_subcat_month_df = fill_missing_year_plus_month_for_subcat(df_psi_subcat_month)

In [ ]:
plot_temp = final_psi_subcat_month_df[(final_psi_subcat_month_df['product_subcategory'] == "conditioner") & (final_psi_subcat_month_df['priority'] == 'p3') &
                                (final_psi_subcat_month_df['warehouse_country'] == 'indonesia') & (final_psi_subcat_month_df['destination'] == 'indonesia')]
plot_temp = plot_temp[-8:] # last 8 months plot
fig = px.line(plot_temp, x="year_plus_month", y="demand_score", title='PSI SubCat Month')
fig.show()

In [ ]:
demand_psi_subcat_month_country_maps = final_psi_subcat_month_df.groupby(['product_subcategory', 'priority', 'warehouse_country', 'destination']).agg({'demand_score' : 'nunique'})\
.reset_index().rename(columns = {'demand_score' : 'unique_demand_scores'})
print(len(demand_psi_subcat_month_country_maps))
demand_psi_subcat_month_country_maps = demand_psi_subcat_month_country_maps[demand_psi_subcat_month_country_maps.unique_demand_scores >= 8] # 8 months

demand_psi_subcat_month_country_maps = demand_psi_subcat_month_country_maps.drop('unique_demand_scores', axis = 1)
print(len(demand_psi_subcat_month_country_maps))

880
634


In [ ]:
# def dump_files_for_inference_pipeline():
#   print('Dumping files for inference later ...')
#   joblib.dump(final_psi_cat_week_df, '/content/drive/MyDrive/PSI_Modeling_Data/PSI/demand/final_psi_cat_week_df.pkl')
#   joblib.dump(final_psi_cat_month_df, '/content/drive/MyDrive/PSI_Modeling_Data/PSI/demand/final_psi_cat_month_df.pkl')
#   joblib.dump(final_psi_subcat_week_df, '/content/drive/MyDrive/PSI_Modeling_Data/PSI/demand/final_psi_subcat_week_df.pkl')
#   joblib.dump(final_psi_subcat_month_df, '/content/drive/MyDrive/PSI_Modeling_Data/PSI/demand/final_psi_subcat_month_df.pkl')
#   joblib.dump(demand_psi_cat_week_country_maps, '/content/drive/MyDrive/PSI_Modeling_Data/PSI/demand/demand_psi_cat_week_country_maps.pkl')
#   joblib.dump(demand_psi_cat_month_country_maps, '/content/drive/MyDrive/PSI_Modeling_Data/PSI/demand/demand_psi_cat_month_country_maps.pkl')
#   joblib.dump(demand_psi_subcat_week_country_maps, '/content/drive/MyDrive/PSI_Modeling_Data/PSI/demand/demand_psi_subcat_week_country_maps.pkl')
#   joblib.dump(demand_psi_subcat_month_country_maps, '/content/drive/MyDrive/PSI_Modeling_Data/PSI/demand/demand_psi_subcat_month_country_maps.pkl')

In [ ]:
# dump_files_for_inference_pipeline()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>